### Домашнее задание № 1 по курсу "MLOps"
#### Линейная регрессия на Scala
##### Автор: Кравченя Павел

##### Цели работы:
Потренироваться в выполнении вычислений на Scala, реализовать алгоритм обучения линейной регрессии на Scala с использованием библиотеки Breeze.

##### Постановка задачи:

1. Скачать данные о комментариях пользователей Facebook ([Facebook Comments датасет](https://www.kaggle.com/kiranraje/prediction-facebook-comment)).

2. Выполнить разведочный анализ датасета. Выбрать подходящую метрику качества (можно взять несколько).
Сделать предобработку признаков для линейной регрессии.

3. Построить модель простой линейной регрессии, используя стандартные функции ``Breeze``. Оценить качество модели.

4. Реализовать свою версию алгоритма линейной регрессии, используя только базовый функционал ``Breeze`` для работы с матрицами (без использования ``breeze.stats.regression`` или других библиотек, реализующих регрессию в готовом виде).

Работа выполнялась с использованием Docker-образа системы ``almond.sh`` с версией ``Scala 2.13``.

Рассматриваемый датасет является сокращенной копией другого датасета, [описание его признаков](https://archive.ics.uci.edu/ml/datasets/Facebook+Comment+Volume+Dataset#) представлено ниже.

* **likes**:     Defines the popularity or support for the source of the document.
* **Checkins**:  Describes how many individuals so far visited this place. This feature is only associated with the places eg:some institution, place, theater etc.
* **Returns**:   Defines the daily interest of individuals towards source of the document / Post. The people who actually come back to the page, after liking the page. This include activities such as comments, likes to a post, shares, etc by visitors to the page.
* **Category**:  Defines the category of the source of the document eg: place, institution, brand etc.
* **commBase**:  The total number of comments before selected base date/time.
* **comm24**:    The number of comments in last 24 hours, relative to base date/time.
* **comm48**:    The number of comments in last 48 hours, relative to base date/time.
* **comm24_1**:  The number of comments in the first 24 hours after the publication of post but before base date/time.
* **diff2448**:  The number of comments in last 48 to last 24 hours relative to base date/time.
* **baseTime**:  Selected time in order to simulate the scenario.
* **length**:    Character count in the post.
* **shares**:    This features counts the № of shares of the post, that how many peoples had shared this post on to their timeline.
* **hrs**:       This describes the H hrs, for which we have the target variable/ comments received.
* **sun_pub,mon_pub,tue_pub,wed_pub,thu_pub,fri_pub,sat_pub**: This represents the day(Sunday...Saturday) on which the post was published.
* **sun_base,mon_base,tue_base,wed_base,thu_base,fri_base,sat_base**: This represents the day(Sunday..Saturday) on selected base Date/Time.

* **output**:   The no of comments in next H hrs

Установим необходимые библиотеки для выполнения вычислений и визуализации.

In [1]:
import $ivy.`org.scalanlp:breeze_2.13:2.0.1-RC1`
import $ivy.`org.scalanlp:breeze-natives_2.13:2.0.1-RC1`
import $ivy.`org.scalanlp:breeze-viz_2.13:2.0.1-RC1`
import $ivy.`org.plotly-scala::plotly-almond:0.8.2`

import $ivy.$                                   

import $ivy.$                                           

import $ivy.$                                       

import $ivy.$                                      

Загрузим необходимые для работы модули.

In [2]:
import scala.collection.mutable.Map
import scala.collection.immutable.ListMap
import scala.collection.mutable.ArrayBuffer
import breeze.linalg._
import breeze.numerics._
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
import breeze.stats.distributions.{Gaussian, ThreadLocalRandomGenerator, RandBasis}
import org.apache.commons.math3.random.MersenneTwister
import breeze.stats.regression.leastSquares

import scala.collection.mutable.Map

import scala.collection.immutable.ListMap

import scala.collection.mutable.ArrayBuffer

import breeze.linalg._

import breeze.numerics._

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import breeze.stats.distributions.{Gaussian, ThreadLocalRandomGenerator, RandBasis}

import org.apache.commons.math3.random.MersenneTwister

import breeze.stats.regression.leastSquares

Данные датасета представлены в ``csv``-формате. Для их чтения и парсинга используется функция, продемонстрированная преподавателем во время вебинара по соответствующей теме. Данная функция была незначительно доработана: при появлении пропущенного значения в матрицу записывается не нулевое значение (как было в изначальной версии), а ``Double.NaN``. Это позволит проанализировать распределение ``NaN``-значений в признаках во время разведочного анализа и впоследствии заменить их более походящими значениями (не обязательно нулевыми).

In [3]:
def read_csv(path: String): DenseMatrix[Double] = {
    val dataFile = io.Source.fromFile(path)
    val dataArray = dataFile.getLines.drop(1)
                    .map(_.split(",").map(_.trim))
                    .map { line => line.map { elem =>
                        elem match {
                            case "" => Double.NaN
                            case x => x.toDouble
                        }
                      }
                    }.toArray

   DenseMatrix(dataArray: _*)
}

defined function read_csv

Читаем файл, формируем матрицу с данными.

In [4]:
val data_matrix = read_csv("Dataset.csv")

data_matrix: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (28 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   NaN   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   NaN   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   NaN   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   NaN   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   NaN   0.0   0.0    52.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   NaN   1.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   NaN   0.0   0.0    3.0   ...
634995.0  0.0  463.0  1.0  1.0   1.0   0.0   1.0   1.0    37.0  ...
63

Для анализа распределения данных необходимо определять количество одинаковых значений признаков. Данная функция реализует расчет числа одинаковых значений в векторе (не считая пропусков) и используется в дальнейшем для этой цели.

In [5]:
def count_distinct_values(df: DenseVector[Double]): Map[Double, Int] = {
    val counter = Map[Double, Int]()
    df.toArray.filter(x => !x.isNaN).foreach( x => if (!counter.contains(x))
                                             counter += (x -> 1)
                                        else
                                             counter.update(x, counter(x) + 1) )
    counter
}

defined function count_distinct_values

Определяем количество различных значений в столбцах матрицы (исключая пропуски).

In [6]:
(0 until data_matrix.cols).map(x => (x, count_distinct_values(data_matrix(::, x)).keys.size))
    .foreach(x => println(s"feature ${x._1} has ${x._2} different non-null values"))

feature 0 has 639 different non-null values
feature 1 has 173 different non-null values
feature 2 has 507 different non-null values
feature 3 has 81 different non-null values
feature 4 has 970 different non-null values
feature 5 has 628 different non-null values
feature 6 has 602 different non-null values
feature 7 has 936 different non-null values
feature 8 has 1092 different non-null values
feature 9 has 73 different non-null values
feature 10 has 1477 different non-null values
feature 11 has 1639 different non-null values
feature 12 has 24 different non-null values
feature 13 has 2 different non-null values
feature 14 has 2 different non-null values
feature 15 has 2 different non-null values
feature 16 has 2 different non-null values
feature 17 has 2 different non-null values
feature 18 has 2 different non-null values
feature 19 has 2 different non-null values
feature 20 has 2 different non-null values
feature 21 has 2 different non-null values
feature 22 has 2 different non-null va

Из представленной информации можно сделать вывод, что признаки 0-2, 4-8, 10, 11 являются количественными, 3, 9, 12 -- категориальными, 13-26 -- бинарными. Это согласуется с информацией о датасете, представленной выше. Целевой признак 27 также является количественным. Здесь и далее для удобства будем разделять количественные и бинарные признаки.

Для упрощения работы с признаками реализуем соответствие между типом признаков и номерами признаков, к нему относящимися:

In [7]:
def features_of_type(feature_type: String): Array[Int] = {
    feature_type match {
        case "quantitative" => Array(0, 1, 2) ++ (4 to 8).toArray ++ Array(10, 11, 27)
        case "categorical"  => Array(3, 9, 12)
        case "binary"       => (13 to 26).toArray
    }
}

defined function features_of_type

Посмотрим, какие значения содержат три категориальных признака:

In [8]:
features_of_type("categorical").map( x => count_distinct_values(data_matrix(::, x)) ).foreach( x => println(x) )

HashMap(1.0 -> 907, 2.0 -> 1050, 3.0 -> 195, 4.0 -> 1110, 5.0 -> 361, 6.0 -> 102, 8.0 -> 1392, 9.0 -> 7494, 10.0 -> 87, 11.0 -> 19, 12.0 -> 203, 13.0 -> 512, 14.0 -> 1684, 15.0 -> 46, 16.0 -> 1890, 17.0 -> 498, 18.0 -> 4301, 19.0 -> 175, 20.0 -> 303, 21.0 -> 183, 22.0 -> 266, 23.0 -> 168, 24.0 -> 4511, 25.0 -> 38, 26.0 -> 501, 27.0 -> 463, 28.0 -> 439, 29.0 -> 59, 30.0 -> 282, 31.0 -> 290, 32.0 -> 1388, 33.0 -> 17, 34.0 -> 263, 35.0 -> 239, 36.0 -> 2387, 38.0 -> 494, 39.0 -> 539, 40.0 -> 584, 42.0 -> 412, 44.0 -> 437, 45.0 -> 111, 46.0 -> 475, 47.0 -> 199, 49.0 -> 116, 50.0 -> 136, 51.0 -> 63, 54.0 -> 119, 55.0 -> 405, 56.0 -> 104, 57.0 -> 121, 58.0 -> 2, 59.0 -> 53, 60.0 -> 224, 61.0 -> 98, 62.0 -> 16, 63.0 -> 4, 66.0 -> 123, 67.0 -> 163, 68.0 -> 166, 72.0 -> 25, 73.0 -> 57, 75.0 -> 67, 76.0 -> 36, 77.0 -> 82, 79.0 -> 79, 80.0 -> 335, 81.0 -> 22, 82.0 -> 24, 83.0 -> 1, 85.0 -> 27, 87.0 -> 141, 89.0 -> 22, 90.0 -> 61, 91.0 -> 188, 92.0 -> 232, 93.0 -> 1, 96.0 -> 89, 100.0 -> 239, 101.0

Проанализируем пропуски в данных. Определим, какие признаки содержат пропущенные значения:

In [9]:
(0 until data_matrix.cols).map( x => 
                               (x, data_matrix(::, x).toArray.count(_.isNaN))
).filter(x => x._2 != 0).foreach(x => println(s"feature ${x._1} has ${x._2} missed values"))

feature 2 has 51 missed values
feature 3 has 57 missed values
feature 4 has 60 missed values
feature 6 has 48 missed values
feature 11 has 2449 missed values
feature 14 has 1927 missed values
feature 17 has 3045 missed values
feature 21 has 1970 missed values


Видно, что пропуски содержатся в четырех количественных признаках (2, 4, 6, 11), в одном категориальном (3) и трех бинарных (14, 17, 21). Линейные модели не могут работать с пропусками, поэтому их нужно заполнить. Будем заполнять пропуски в количественных признаках медианным значением, для пропусков в категориальном признаке добавим новую категорию, а пропущенные значения в бинарных признаках заменим числом 0.5. Как можно видеть из предыдущей ячейки, третий признак не содержит нулевых значений. Поэтому, все пропуски в третьем столбце можно заменить нулями, обозначающими катерогию с отсутствующими значениями столбца.

Сначала обработаем количественные признаки. Заменим пропуски в данных на медианное значение. Реализуем функцию расчета квантилей.

In [10]:
def quartile(data: DenseVector[Double], q: Double): Double = {
    val sorted_data = data.toArray.sorted
    val pos: Double = q * (sorted_data.size - 1)
    val base: Int = pos.floor.toInt
    val rest: Double = pos - base
    
    if (base < sorted_data.size - 1) {
        sorted_data(base) + rest * (sorted_data(base + 1) - sorted_data(base))
    } else {
        sorted_data(base)
    }
}

defined function quartile

In [11]:
def convert_nan_quantitative(data: DenseVector[Double]): DenseVector[Double] = {
    val substitute_value = quartile( DenseVector(data.toArray.filter(x => !x.isNaN): _*), 0.5)
    data.map(x => if (x.isNaN) substitute_value else x)
}

defined function convert_nan_quantitative

Реализуем функцию замены категориального признака нулевым значением (соответствующим отсутствию категории).

In [12]:
def convert_nan_categorical(data: DenseVector[Double]): DenseVector[Double] = {
    data.map(x => if (x.isNaN) 0.0 else x)
}

defined function convert_nan_categorical

Реализуем функцию замены бинарных признаков значением "неопределенности" -- 0.5.

In [13]:
def convert_nan_binary(data: DenseVector[Double]): DenseVector[Double] = {
    data.map(x => if (x.isNaN) 0.5 else x)
}

defined function convert_nan_binary

Для удобной работы с обработкой признаков определим вспомогательную функцию, которая будет применять некоторые преобразования, определенные функцией над вектором, поочередно к заданному множеству векторов признаков.

In [14]:
def transform(data: DenseMatrix[Double], 
              featutes_array: Array[Int],
              transform_func: DenseVector[Double] => DenseVector[Double]): DenseMatrix[Double] = {
    val aux_data = DenseMatrix.zeros[Double](data.rows, data.cols)
    aux_data := data
    
    for (x <- featutes_array) {
        aux_data(::, x) := transform_func(data(::, x))
    }
    aux_data
}

defined function transform

С использованием вышеопределенной функции выполним удаление нулевых значений в данных в зависимости от типов признаков.

In [15]:
val data_XY = data_matrix
data_XY := transform(data_XY, features_of_type("quantitative"), convert_nan_quantitative)
data_XY := transform(data_XY, features_of_type("categorical"), convert_nan_categorical)
data_XY := transform(data_XY, features_of_type("binary"), convert_nan_binary)

data_XY: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (28 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    52.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    3.0   ...
634995.0  0.0  463.0  1.0  1.0   1.0   0.0   1.0   1.0    37.0  ...
634995

Построим гистрограмму значений количественных признаков, чтобы оценить имеющиеся в них данные и их распределение.

In [16]:
def plot_vector_distribution(data: DenseVector[Double], 
                             feature_num: Int,
                             bins_count: Int = 150): Unit = {
    val delta = (max(data) - min(data)) / bins_count
    val data_min = min(data)
    val counter = Map[Int, Int]()
    
    (0 to bins_count).foreach( x => counter += (x -> 0) )
    data.foreach( x => { 
        val ind = ((x - data_min) / delta).floor.toInt
        counter.update(ind, counter(ind) + 1)
    })
    
    val counts = counter.map( x => (data_min + x._1 * delta, x._2) )
    val distr = ListMap(counts.toSeq.sortBy(_._1):_*)

    val plot_data = Seq(
        Bar(
            distr.map( x => x._1).toSeq,
            distr.map( x => x._2).toSeq
        )
    )

    val plot_layout = Layout(
      title = s"Distibution of feature $feature_num"
    )

    plot(plot_data, plot_layout)
}

defined function plot_vector_distribution

In [17]:
features_of_type("quantitative").foreach( x => plot_vector_distribution(data_XY(::,x), x) )

Из диаграмм видно, что:
1. Распределение значений количественных признаков сильно отличается от нормального.
2. В данных имеются выбросы.
3. Масштабы изменения значений различных признаков существенно различаются.

Для обработки таких признаков для каждого из них построим отрезок [Q1 - 3.0 IQR, Q3 + 3.0 IQR]. Все значения, которые не лежат в данном отрезке, будем считать выбросами, и заменим их на крайнее значение отрезка. После этой операции все значения признаков будут принадлежать рассматриваемому отрезку. Затем произведем масштабирование отрезка на [0, 1] MinMax-методом.

In [18]:
def clipping_and_scaling(data: DenseVector[Double]): DenseVector[Double] = {
  
    val coeff = 3.0
    val IQR = quartile(data, 0.75) - quartile(data, 0.25)
    val outliers_min_value = quartile(data, 0.25) - coeff * IQR
    val outliers_max_value = quartile(data, 0.75) + coeff * IQR
    
    val q3 = quartile(data, 0.75)
    val q1 = quartile(data, 0.25)
    
    val new_data = DenseVector.zeros[Double](data.size)
    new_data := data
    new_data(data >:> outliers_max_value) := outliers_max_value
    new_data(data <:< outliers_min_value) := outliers_min_value
    
    (new_data - min(new_data)) / (max(new_data) - min(new_data)) 
}

defined function clipping_and_scaling

In [19]:
data_XY := transform(data_XY, features_of_type("quantitative"), clipping_and_scaling)

res18: DenseMatrix[Double] = 0.1349076980541017  0.0  0.002327182801968304  1.0  ... (28 total)
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.1349076980541017  0.0  0.002327182801968304  1.0  ...
0.134907

Вновь построим диаграммы распределений для преобразованных и отмасштабированных количественных признаков.

In [20]:
features_of_type("quantitative").foreach( x => plot_vector_distribution(data_XY(::,x), x) )

Из гистограмм видно, что область значений признаков стала одинаковой, а распределения отличны от нулевых и околонулевых значений в большей части области изменения признаков, чем до их масштабирования.

Теперь преобразуем категориальные признаки (3, 9, 12) во множество бинарных признаков, характеризующих принадлежность к определенной категории.

In [21]:
def categorical_to_binary(data: DenseVector[Double]): DenseMatrix[Double] = {
    
    def one_hot_encoding(index: Int, depth: Int): DenseVector[Double] = {
        val one_hot = DenseVector.zeros[Double](depth)
        one_hot(index) = 1.0
        one_hot
    }
    
    val depth = count_distinct_values(data).keys.size
    val categorials = count_distinct_values(data).keys.zipWithIndex.toMap
    val matrix = DenseMatrix.zeros[Double](data.size, depth)
    
    (0 until data.size).foreach(x => matrix(x, ::) := one_hot_encoding(categorials(data(x)), depth).t)
    matrix
}

defined function categorical_to_binary

Реализуем функцию замены столбца категориального признака в матрице на множество столбцов с бинарным значением принадлежности признака к определенной категории. Новые столбцы присоединяются справа, а столбец с категориальным признаком удаляется.

In [22]:
def convert_categorical(data: DenseMatrix[Double]): DenseMatrix[Double] = {
    var aux_matrix = DenseMatrix.zeros[Double](data.rows, data.cols - 1)
    aux_matrix := data(::, 0 to -2)
    
    features_of_type("categorical").foreach( x => {
        aux_matrix = DenseMatrix.horzcat(aux_matrix, categorical_to_binary( data(::, x) ))
    })
    val matrix = aux_matrix.delete(Array(3, 9, 12), breeze.linalg.Axis._1)
    matrix
}

defined function convert_categorical

Теперь выполняем само преобразование категориальных признаков, одновременно разделяя данные для обучения и целевой вектор.

In [23]:
val data_X = convert_categorical(data_XY)
val data_Y = data_XY(::, -1)

data_X: DenseMatrix[Double] = 0.1349076980541017  0.0  0.002327182801968304  ... (203 total)
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.002327182801968304  ...
0.1349076980541017  0.0  0.0023271828019

Для удобства работы с матрицей "объекты-признаки" и целевым вектором объединим их в единый класс.

In [24]:
class Dataset(data_X: DenseMatrix[Double],
              data_Y: DenseVector[Double]) {
    
    private val _objects_features: DenseMatrix[Double] = data_X
    private val _target_vector: DenseVector[Double] = data_Y
    
    def X = _objects_features
    def Y = _target_vector 
}

defined class Dataset

Создаем объект датасета и инициализируем его подготовленными значениями признаков и целевого вектора.

In [25]:
val dataset = new Dataset(data_X, data_Y)

dataset: Dataset = ammonite.$sess.cmd23$Helper$Dataset@2c02eccd

Реализуем функцию для перемешивания объектов в датасете и разделения его на обучающую и тестовую части. Перемешивание реализовывается следующим образом: в цикле с достаточно большим количеством повторений на каждой итерации выбираются две произвольные строки матрицы и меняются местами. На каждой итерации равновероятно выбираются различные строки.

In [26]:
def shuffle_and_split_dataset(dataset: Dataset, ratio: Double): (Dataset, Dataset) = {
    val matrix = DenseMatrix.horzcat(dataset.X, dataset.Y.asDenseMatrix.t)
    val rnd = new scala.util.Random
    val temp = DenseVector.zeros[Double](matrix.cols).t
    
    (0 to matrix.rows).foreach(x => {
        val a = rnd.nextInt( matrix.rows )
        val b = rnd.nextInt( matrix.rows )
        temp := matrix(b, ::)
        matrix(b, ::) := matrix(a, ::)
        matrix(a, ::) := temp
    })
    
    val sep_value = (ratio * matrix.rows).toInt
    val train_matrix = matrix(0 until sep_value, ::)
    val test_matrix = matrix(sep_value until matrix.rows, ::)
   
    (new Dataset(train_matrix(::, 0 to -2), train_matrix(::, -1)), 
     new Dataset(test_matrix(::, 0 to -2), test_matrix(::, -1)))
}

defined function shuffle_and_split_dataset

In [27]:
val (train_dataset, test_dataset) = shuffle_and_split_dataset(dataset, 0.8)

train_dataset: Dataset = ammonite.$sess.cmd23$Helper$Dataset@1f59f4e
test_dataset: Dataset = ammonite.$sess.cmd23$Helper$Dataset@32368249

В соответствии с постановкой задачи, напишем свой собственный класс, реализующий линейную регрессию. Линейная регрессия выражается соотношением:
$$y = Xw + w_0 \mathbb{I}$$
где $$y \in \mathbb{R}^n, X \in \mathbb{R}^{n \times m}, w \in \mathbb{R}^m, w_0 \in \mathbb{R}, \mathbb{I} = \{1\}_{i=1}^{n}$$

Здесь $X$ -- матрица "объекты-признаки", $y$ -- целевой вектор, $w$, $w_0$ -- параметры модели, $n$ -- количество объектов, $m$ -- количество признаков.

Обучение модели линейной регрессии будем выполнять градиентным спуском. Данный метод предполагает вычисления векторов производных функции ошибки от весов модели $\frac{\partial L}{\partial w}$ и $\frac{\partial L}{\partial w_0}$. Выразим их:

$$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \frac{\partial y}{\partial w}$$
$$\frac{\partial L}{\partial w_0} = \frac{\partial L}{\partial y} \frac{\partial y}{\partial w_0}$$

В качестве функции ошибки возьмем MSE: 
$$L(y,t) = \frac{1}{2} ||y-t||_2^2 = \frac{1}{2n} \sum_{i=1}^n {(y^i-t^i)}^2$$

где $t$ -- истинное значение, а $y$ -- результат, предсказанный моделью. Тогда производные по $y$ и $w$ выразятся в виде:

$$\left[ \frac{\partial L}{\partial y} \right]_{i} = \frac{\partial L}{\partial y_i} = \frac{\partial}{\partial y_i} \frac{1}{2n} \sum_{j=1}^n {(y^j-t^j)}^2 = \frac{1}{n} \sum_{j=1}^n {(y^j-t^j) \frac{\partial y^i}{\partial y_j}} = \frac{1}{n} (y-t)^i \quad \Rightarrow \quad \frac{\partial L}{\partial y} = (y - t)^T$$

$$\left[ \frac{\partial y}{\partial w} \right]_j^i = \frac{\partial y^i}{\partial w_j} = \frac{\partial}{\partial w_j} \left( Xw + w_0 \mathbb{I} \right)^i = \frac{\partial}{\partial w_j} \left( \sum_{k=1}^n {X_k^i w^k} + w_0 \right) = \sum_{k=1}^n {X_k^i \frac{\partial w^k}{\partial w_j}} = X_k^i \quad \Rightarrow \quad \frac{\partial y}{\partial w} = X$$

$$\left[ \frac{\partial y}{\partial w_0} \right]^i = \frac{\partial y^i}{\partial w_0} = \frac{\partial}{\partial w_0} \left( Xw + w_0 \mathbb{I} \right)^i = \frac{\partial}{\partial w_0} \left( \sum_{k=1}^n {X_k^i w^k} + w_0 \right) = 1  \quad \Rightarrow \quad \frac{\partial y}{\partial w_0} = \mathbb{I}$$

Теперь можно выразить прозводные от функции ошибки по весам модели:

$$\frac{\partial L}{\partial w} = (y - t)^T X$$

$$\frac{\partial L}{\partial w_0} = (y - t)^T \cdot \mathbb{I} = \sum_{i=1}^n{\left( y-t \right)^j}$$

Данные выражения для градиентов используются в методе градиентного спуска (и во всех его модификациях). Для ускорения сходимости решения в работе применяется [модификация Adam](https://arxiv.org/abs/1412.6980) градиентного спуска. Также, в классе необходимо реализовать методы расчета значений ошибки и метрик, а также основной цикл обучения.

В качестве метрик качества были выбраны коэффициент детерминации $R^2$ и квадратный корень из среднеквадратичной ошибки ``RMSE``.

In [28]:
class LinearRegressor(val features_count: Int, val learning_rate: Double = 1E-3) {
    // Параметры нормального распределения для инициализации весов
    private val seed = 42
    private val randBasis: RandBasis = new RandBasis(new ThreadLocalRandomGenerator(new MersenneTwister(seed)))
    private val normal_distr = breeze.stats.distributions.Gaussian(0, 1)(randBasis)
    
    // Инициализация весов модели
    private var w  = DenseVector.rand(features_count, normal_distr)
    private var w0: Double = normal_distr.sample()
    
    // Определение и инициализация параметров для Adam
    private val m = DenseVector.zeros[Double](features_count)
    private var m0: Double = 0.0
    private val v = DenseVector.zeros[Double](features_count)
    private var v0: Double = 0.0
    private val beta1 = 0.9
    private val beta2 = 0.999
    
    // Функция применения модели регрессии к входным данным
    def forward(X: DenseMatrix[Double]): DenseVector[Double] = { X * w + w0 }
    
    // Функция ошибки MSE
    def loss(X: DenseMatrix[Double], t: DenseVector[Double]): Double = {
        val y = forward(X)
        0.5 / t.size * sum( (y - t) *:* (y - t) )
    }
    
    // Метрика -- коэффициент детерминации
    def r2(X: DenseMatrix[Double], t: DenseVector[Double]): Double = {
        val y = forward(X)
        val length = t.size
        1 - (1.0 / length) * sum((y - t) *:* (y - t)) / breeze.stats.meanAndVariance(y).variance
    }
    
    // Функция реализации одного шага стохастического градиентного спуска
    def step_sgd(X: DenseMatrix[Double],
             t: DenseVector[Double]): Unit = {
        val y = forward(X)
        w  = w  - learning_rate / t.size * X.t * (y - t)
        w0 = w0 - learning_rate / t.size * sum(y - t)
    }
    
    // Функция реализации одного шага метода Adam
    def step_adam(X: DenseMatrix[Double],
                  t: DenseVector[Double],
                  step: Int): Unit = {
        val y = forward(X)
        
        val grads  = 1.0 / t.size * X.t * (y - t)
        val grads0 = 1.0 / t.size * sum(y - t)
        
        m := beta1 * m  + (1.0 - beta1) * grads
        m0 = beta1 * m0 + (1.0 - beta1) * grads0
        
        v := beta2 * v  + (1.0 - beta2) * grads *:* grads
        v0 = beta2 * v0 + (1.0 - beta2) * grads0 * grads0
        
        val m_tilda  = m  / (1.0 - scala.math.pow(beta1, step))
        val m0_tilda = m0 / (1.0 - scala.math.pow(beta1, step))
        
        val v_tilda  = v  / (1.0 - scala.math.pow(beta2, step))
        val v0_tilda = v0 / (1.0 - scala.math.pow(beta2, step))
        
        w  = w  - learning_rate * m_tilda /:/ (sqrt(v_tilda) + 1E-10)
        w0 = w0 - learning_rate * m0_tilda / (sqrt(v0_tilda) + 1E-10)
    }
    
    // Основной цикл обучения и валидации модели
    def fit(train_dataset: Dataset, test_dataset:Dataset, num_steps: Int): Map[String, ArrayBuffer[Double]] = {
        var train_loss_arr = ArrayBuffer[Double]()
        var test_loss_arr  = ArrayBuffer[Double]()
        var r2_arr         = ArrayBuffer[Double]()
        var rmse_arr       = ArrayBuffer[Double]()
        
        (1 to num_steps).foreach(x => {
            step_adam(train_dataset.X, train_dataset.Y, x)
            val train_loss = loss(train_dataset.X, train_dataset.Y)
            train_loss_arr += train_loss
            
            val test_loss = loss(test_dataset.X, test_dataset.Y)
            test_loss_arr += test_loss
            
            val rmse_metric = sqrt(test_loss)
            rmse_arr += rmse_metric
            
            val r2_metric = r2(test_dataset.X, test_dataset.Y)
            r2_arr += r2_metric
            
            println(f"Step: $x%4.0f    train loss: $train_loss%2.8f    test loss: $test_loss%2.8f    RMSE: $rmse_metric%2.8f    R^2: $r2_metric%1.4f")       
        })
        
        Map("Train Loss" -> train_loss_arr,
            "Test Loss"  -> test_loss_arr,
            "RMSE"       -> rmse_arr,
            "R^2"        -> r2_arr
            )
    }
}

defined class LinearRegressor

Реализуем функции для визуализации графиков зависимости значений функции ошибки (на обучающем и валидационном примерах) и метрик от номера шага расчета.

In [29]:
def plot_losses(train_loss: ArrayBuffer[Double], test_loss: ArrayBuffer[Double]): Unit = {
    val plot_data = Seq(
        Scatter(
            (0 until train_loss.size).toSeq,
            train_loss.toSeq,
            name = "Train MSE-loss",
            mode = ScatterMode(ScatterMode.Lines)
        ),
        
        Scatter(
            (0 until test_loss.size).toSeq,
            test_loss.toSeq,
            name = "Test MSE-loss",
            mode = ScatterMode(ScatterMode.Lines)
        )
    )    

    val plot_layout = Layout(
      title = s"Process of training and validation"
    )

    plot(plot_data, plot_layout)
}

defined function plot_losses

In [30]:
def plot_metrics(data: Map[String, ArrayBuffer[Double]]): Unit = {
    val plot_data = Seq(
        Scatter(
            (0 until data("R^2").size).toSeq,
            data("R^2").toSeq,
            name = "R2",
            mode = ScatterMode(ScatterMode.Lines)
        ),
        Scatter(
            (0 until data("RMSE").size).toSeq,
            data("RMSE").toSeq,
            name = "RMSE",
            mode = ScatterMode(ScatterMode.Lines)
        )
    )    

    val plot_layout = Layout(
      title = s"Metrics of validation"
    )

    plot(plot_data, plot_layout)
}

defined function plot_metrics

Создаем объект класса линейной регрессии.

In [31]:
val regressor = new LinearRegressor(train_dataset.X.cols, learning_rate = 1E-2)

regressor: LinearRegressor = ammonite.$sess.cmd27$Helper$LinearRegressor@19029872

Запускаем цикл расчета и валидации модели. Количество шагов расчета задается явно в параметре вызова метода.

In [32]:
val metrics = regressor.fit(train_dataset, test_dataset, num_steps = 700)

Jan 29, 2022 10:17:58 PM dev.ludovic.netlib.InstanceBuilder$NativeBLAS getInstanceImpl
Jan 29, 2022 10:17:58 PM dev.ludovic.netlib.InstanceBuilder$NativeBLAS getInstanceImpl
Jan 29, 2022 10:17:58 PM dev.ludovic.netlib.InstanceBuilder$BLAS getInstanceImpl


Step:    1    train loss: 2.93236317    test loss: 2.92748202    RMSE: 1.71098861    R^2: -0.1923
Step:    2    train loss: 2.84036467    test loss: 2.83517732    RMSE: 1.68379848    R^2: -0.1701
Step:    3    train loss: 2.75347228    test loss: 2.74796003    RMSE: 1.65769721    R^2: -0.1494
Step:    4    train loss: 2.67169978    test loss: 2.66586183    RMSE: 1.63274671    R^2: -0.1302
Step:    5    train loss: 2.59501380    test loss: 2.58885853    RMSE: 1.60899302    R^2: -0.1125
Step:    6    train loss: 2.52332706    test loss: 2.51686743    RMSE: 1.58646381    R^2: -0.0963
Step:    7    train loss: 2.45650073    test loss: 2.44974974    RMSE: 1.56516764    R^2: -0.0818
Step:    8    train loss: 2.39434942    test loss: 2.38731755    RMSE: 1.54509467    R^2: -0.0689
Step:    9    train loss: 2.33664539    test loss: 2.32934053    RMSE: 1.52621772    R^2: -0.0576
Step:   10    train loss: 2.28312021    test loss: 2.27554924    RMSE: 1.50849237    R^2: -0.0479
Step:   11    train 

Step:   84    train loss: 0.59677539    test loss: 0.58899820    RMSE: 0.76746218    R^2: 0.0184
Step:   85    train loss: 0.58573609    test loss: 0.57796256    RMSE: 0.76023849    R^2: 0.0193
Step:   86    train loss: 0.57489884    test loss: 0.56713004    RMSE: 0.75308037    R^2: 0.0202
Step:   87    train loss: 0.56425999    test loss: 0.55649703    RMSE: 0.74598729    R^2: 0.0212
Step:   88    train loss: 0.55381590    test loss: 0.54605996    RMSE: 0.73895870    R^2: 0.0221
Step:   89    train loss: 0.54356297    test loss: 0.53581530    RMSE: 0.73199406    R^2: 0.0231
Step:   90    train loss: 0.53349774    test loss: 0.52575960    RMSE: 0.72509282    R^2: 0.0240
Step:   91    train loss: 0.52361683    test loss: 0.51588952    RMSE: 0.71825449    R^2: 0.0250
Step:   92    train loss: 0.51391702    test loss: 0.50620186    RMSE: 0.71147864    R^2: 0.0260
Step:   93    train loss: 0.50439522    test loss: 0.49669354    RMSE: 0.70476488    R^2: 0.0271
Step:   94    train loss: 0.49

Step:  168    train loss: 0.12976587    test loss: 0.12519961    RMSE: 0.35383558    R^2: 0.1692
Step:  169    train loss: 0.12759940    test loss: 0.12308192    RMSE: 0.35083032    R^2: 0.1720
Step:  170    train loss: 0.12547529    test loss: 0.12100634    RMSE: 0.34785965    R^2: 0.1749
Step:  171    train loss: 0.12339271    test loss: 0.11897203    RMSE: 0.34492322    R^2: 0.1777
Step:  172    train loss: 0.12135083    test loss: 0.11697815    RMSE: 0.34202069    R^2: 0.1807
Step:  173    train loss: 0.11934885    test loss: 0.11502389    RMSE: 0.33915172    R^2: 0.1836
Step:  174    train loss: 0.11738595    test loss: 0.11310844    RMSE: 0.33631598    R^2: 0.1865
Step:  175    train loss: 0.11546137    test loss: 0.11123102    RMSE: 0.33351314    R^2: 0.1895
Step:  176    train loss: 0.11357434    test loss: 0.10939084    RMSE: 0.33074287    R^2: 0.1925
Step:  177    train loss: 0.11172411    test loss: 0.10758717    RMSE: 0.32800483    R^2: 0.1955
Step:  178    train loss: 0.10

Step:  252    train loss: 0.03927941    test loss: 0.03764283    RMSE: 0.19401760    R^2: 0.4324
Step:  253    train loss: 0.03884634    test loss: 0.03723056    RMSE: 0.19295222    R^2: 0.4352
Step:  254    train loss: 0.03842116    test loss: 0.03682591    RMSE: 0.19190078    R^2: 0.4380
Step:  255    train loss: 0.03800373    test loss: 0.03642873    RMSE: 0.19086311    R^2: 0.4407
Step:  256    train loss: 0.03759389    test loss: 0.03603887    RMSE: 0.18983906    R^2: 0.4435
Step:  257    train loss: 0.03719149    test loss: 0.03565619    RMSE: 0.18882847    R^2: 0.4462
Step:  258    train loss: 0.03679641    test loss: 0.03528055    RMSE: 0.18783118    R^2: 0.4489
Step:  259    train loss: 0.03640849    test loss: 0.03491182    RMSE: 0.18684704    R^2: 0.4516
Step:  260    train loss: 0.03602759    test loss: 0.03454985    RMSE: 0.18587589    R^2: 0.4542
Step:  261    train loss: 0.03565359    test loss: 0.03419451    RMSE: 0.18491758    R^2: 0.4569
Step:  262    train loss: 0.03

...

Step:  420    train loss: 0.01578523    test loss: 0.01546305    RMSE: 0.12435050    R^2: 0.6473
Step:  421    train loss: 0.01575989    test loss: 0.01543940    RMSE: 0.12425538    R^2: 0.6476
Step:  422    train loss: 0.01573494    test loss: 0.01541612    RMSE: 0.12416166    R^2: 0.6479
Step:  423    train loss: 0.01571037    test loss: 0.01539320    RMSE: 0.12406933    R^2: 0.6482
Step:  424    train loss: 0.01568619    test loss: 0.01537064    RMSE: 0.12397837    R^2: 0.6485
Step:  425    train loss: 0.01566238    test loss: 0.01534842    RMSE: 0.12388876    R^2: 0.6488
Step:  426    train loss: 0.01563893    test loss: 0.01532656    RMSE: 0.12380047    R^2: 0.6491
Step:  427    train loss: 0.01561585    test loss: 0.01530503    RMSE: 0.12371350    R^2: 0.6493
Step:  428    train loss: 0.01559312    test loss: 0.01528383    RMSE: 0.12362781    R^2: 0.6496
Step:  429    train loss: 0.01557075    test loss: 0.01526297    RMSE: 0.12354339    R^2: 0.6499
Step:  430    train loss: 0.01

Step:  504    train loss: 0.01457828    test loss: 0.01434029    RMSE: 0.11975095    R^2: 0.6614
Step:  505    train loss: 0.01457114    test loss: 0.01433369    RMSE: 0.11972340    R^2: 0.6615
Step:  506    train loss: 0.01456411    test loss: 0.01432719    RMSE: 0.11969625    R^2: 0.6616
Step:  507    train loss: 0.01455717    test loss: 0.01432079    RMSE: 0.11966948    R^2: 0.6616
Step:  508    train loss: 0.01455034    test loss: 0.01431447    RMSE: 0.11964310    R^2: 0.6617
Step:  509    train loss: 0.01454361    test loss: 0.01430825    RMSE: 0.11961710    R^2: 0.6618
Step:  510    train loss: 0.01453697    test loss: 0.01430212    RMSE: 0.11959146    R^2: 0.6618
Step:  511    train loss: 0.01453043    test loss: 0.01429608    RMSE: 0.11956620    R^2: 0.6619
Step:  512    train loss: 0.01452398    test loss: 0.01429012    RMSE: 0.11954129    R^2: 0.6620
Step:  513    train loss: 0.01451762    test loss: 0.01428425    RMSE: 0.11951674    R^2: 0.6620
Step:  514    train loss: 0.01

Step:  588    train loss: 0.01422364    test loss: 0.01401438    RMSE: 0.11838236    R^2: 0.6647
Step:  589    train loss: 0.01422137    test loss: 0.01401232    RMSE: 0.11837363    R^2: 0.6647
Step:  590    train loss: 0.01421913    test loss: 0.01401028    RMSE: 0.11836501    R^2: 0.6647
Step:  591    train loss: 0.01421692    test loss: 0.01400826    RMSE: 0.11835650    R^2: 0.6647
Step:  592    train loss: 0.01421473    test loss: 0.01400627    RMSE: 0.11834810    R^2: 0.6648
Step:  593    train loss: 0.01421257    test loss: 0.01400431    RMSE: 0.11833980    R^2: 0.6648
Step:  594    train loss: 0.01421044    test loss: 0.01400237    RMSE: 0.11833160    R^2: 0.6648
Step:  595    train loss: 0.01420833    test loss: 0.01400045    RMSE: 0.11832351    R^2: 0.6648
Step:  596    train loss: 0.01420625    test loss: 0.01399856    RMSE: 0.11831551    R^2: 0.6648
Step:  597    train loss: 0.01420420    test loss: 0.01399669    RMSE: 0.11830762    R^2: 0.6648
Step:  598    train loss: 0.01

Step:  672    train loss: 0.01410093    test loss: 0.01390332    RMSE: 0.11791234    R^2: 0.6654
Step:  673    train loss: 0.01410002    test loss: 0.01390250    RMSE: 0.11790886    R^2: 0.6654
Step:  674    train loss: 0.01409912    test loss: 0.01390169    RMSE: 0.11790541    R^2: 0.6654
Step:  675    train loss: 0.01409822    test loss: 0.01390088    RMSE: 0.11790199    R^2: 0.6654
Step:  676    train loss: 0.01409733    test loss: 0.01390008    RMSE: 0.11789861    R^2: 0.6654
Step:  677    train loss: 0.01409645    test loss: 0.01389929    RMSE: 0.11789525    R^2: 0.6654
Step:  678    train loss: 0.01409558    test loss: 0.01389850    RMSE: 0.11789192    R^2: 0.6654
Step:  679    train loss: 0.01409471    test loss: 0.01389773    RMSE: 0.11788861    R^2: 0.6654
Step:  680    train loss: 0.01409385    test loss: 0.01389695    RMSE: 0.11788534    R^2: 0.6654
Step:  681    train loss: 0.01409300    test loss: 0.01389619    RMSE: 0.11788210    R^2: 0.6654
Step:  682    train loss: 0.01

metrics: Map[String, ArrayBuffer[Double]] = HashMap(
  "Test Loss" -> ArrayBuffer(
    2.9274820187054487,
    2.8351773200125594,
    2.747960032356141,
    2.665861834562611,
    2.588858526817524,
    2.5168674291623465,
    2.449749738328696,
    2.3873175542501297,
    2.3293405278159223,
    2.275549243940474,
    2.225638743017794,
    2.179274602698517,
    2.1361005082075315,
    2.0957459754664516,
    2.0578341426096087,
    2.02199016915482,
    1.9878504721995285,
    1.9550723834977939,
    1.9233433573530097,
    1.8923887648693845,
    1.86197750777335,
    1.8319250329461505,
    1.8020936965719552,
    1.77239072772645,
    1.7427642370814536,
    1.713197806942167,
    1.6837042055377995,
    1.6543187198777678,
    1.6250925236815046,
    1.5960864094036185,
    1.5673651285159667,
    1.5389925083065272,
    1.511027448641628,
    1.4835208480277324,
    1.4565134633183914,
    1.4300346696870427,
    1.4041020554917945,
...

После обучения модели выполняем визуализацию кривых обучения / валидации и значений метрик.

In [33]:
plot_losses(metrics("Train Loss"), metrics("Test Loss"))

In [34]:
plot_metrics(metrics)

Из графиков видно, что примерно к 500-му шагу модель достигает наилучших значений метрик, которые при дальнейшем обучении модели не улучшаются.

Теперь, в соответствии с постановкой задачи, изучим модель линейной регрессии, реализованной посредством встроенных классов библиотеки Scala Breeze. Анализ [учебной литературы](https://books.google.ru/books?id=CVQoDwAAQBAJ&pg=PA57&lpg=PA57&dq=breeze+matrix+regression&source=bl&ots=-j_ulO5j-5&sig=ACfU3U2Q3FLQZfyeoyxJi9oD6Q5lzF1dhg&hl=ru&sa=X&ved=2ahUKEwjujvD7gNf1AhWRy4sKHYVaAToQ6AF6BAgHEAM#v=onepage&q=breeze%20matrix%20regression&f=false) (а также сведений в [открытых источниках](https://github.com/scalanlp/breeze/tree/master/math/src/main/scala/breeze/stats/regression), в Интернете) показал, что фактически единственным способом реализации линейной регрессии в Breeze является объект ``leastSquares`` пакета ``breeze.stats.regression``. Но он выполняет регрессию над одномерными данными с их метками, которые можно расположить на плоскости, и рассчитывает для них коэффициенты аппроксимирующей прямой. В нашем случае, каждый объект данных имеет несколько признаков, что затрудняет непосредственное использование ``leastSquares``. Поэтому, для его применения с такими ограничениями было принято решение определить признак в данных, который сильнее всего коррелирует с целевым вектором, и построить регресию только для него. Корреляцию проще всего определить, рассчитав коэффициенты корреляции Пирсона целевого вектора со всеми векторами признаков и выбрав наибольший. Вектор признаков, ему соответствующий, и будет кандидатом на использование в линейной регрессии ``leastSquares``.

Для решения этой задачи составим функцию расчета коэффициента корреляции Пирсона для двух векторов.

In [35]:
def corr(a: DenseVector[Double], b: DenseVector[Double]): Double = {
    val length = a.length
    
    val amv = breeze.stats.meanAndVariance(a)
    val bmv = breeze.stats.meanAndVariance(b)
    
    val astddev = math.sqrt(amv.variance)
    val bstddev = math.sqrt(bmv.variance)
    
    1.0 / (length - 1.0) * sum( ((a - amv.mean) / astddev) *:* ((b - bmv.mean) / bstddev) )
} 

defined function corr

Реализуем функцию для определения величины корреляции целевого вектора с векторами признаков. Функция будет выводить номера признаков и значение коээфициента Пирсона, если оно больше 0.7 (т.е., между векторами наблюдается линейная зависимость).

In [36]:
def print_corr_target_with_features(data: DenseMatrix[Double],
                                    target: DenseVector[Double]): Unit = {
    (0 until data.cols).foreach(x => {
        val res_corr = corr( data(::, x), target )
        if (abs(res_corr) > 0.7) {
            println(s"feature[$x] = $res_corr")
        }
    })
}

defined function print_corr_target_with_features

In [37]:
print_corr_target_with_features(dataset.X, dataset.Y)

feature[4] = 0.794603055036997


Видно, что единственным признаком, имеющим хорошую корреляцию с целевым вектором, является признак 4. Согласно описанию датасета, данный признак соответствует полному количеству комментариев перед выбранными базовыми датой / временем. Целевой вектор, в свою очередь, выражает количество комментариев в последующие ``H`` часов. Корреляция между ними вполне естественна. Этот признак и будем в дальнейшем использовать для построения регрессии.

Для использования данных при выполнении линейной регрессии в ``leastSquares`` их необходимо соответствующим образом подготовить. Данный объект принимает входные данные в виде матрицы, в которой первый столбец содержит единицы, а второй соответствует компонентам вектора выбранного признака.

In [38]:
def train_data_for_leastSquares(data: DenseVector[Double]): DenseMatrix[Double] = {
    DenseMatrix.horzcat(
        DenseMatrix.ones[Double](data.size, 1),
        data.asDenseMatrix.t
    )
}

defined function train_data_for_leastSquares

Создадим и обучим модель линейной регрессии, представленную объектом ``breeze.stats.regression.leastSquares``.

In [39]:
val result = leastSquares(train_data_for_leastSquares(train_dataset.X(::, 4)),
                          train_dataset.Y)

println(s"intercept = ${result.coefficients.data(0)}")
println(s"slope = ${result.coefficients.data(1)}")
println(s"r^2 = ${result.rSquared}")

Jan 29, 2022 10:21:04 PM dev.ludovic.netlib.InstanceBuilder$NativeLAPACK getInstanceImpl
Jan 29, 2022 10:21:04 PM dev.ludovic.netlib.InstanceBuilder$LAPACK getInstanceImpl


intercept = 0.03265205463391061
slope = 0.8016725176208309
r^2 = 1317.2226169490323


result: breeze.stats.regression.LeastSquaresRegressionResult = LeastSquaresRegressionResult(
  DenseVector(0.03265205463391061, 0.8016725176208309),
  1317.2226169490323
)

Можно видеть, что объект ``leastSquares`` рассчитал коэффициенты аппроксимирующей прямой. Однако, значение коэффициента детерминации $R^2$ вызывает недоумение. Данная метрика по определению не может быть больше единицы. Поиск аналогичной проблемы в Интернете [показал](https://stackoverflow.com/questions/38350173/fitting-linear-model-in-scalanlp-breeze), что подобный ответ вызван ошибкой в коде библиотеки, которая до сих пор не исправлена разработчиками.

Попробуем вычислить значения $R^2$ и ``RMSE`` вручную, ведь параметры модели теперь известны. Вычислим вектор предсказанных моделью значений:

In [40]:
val simple_model_predictions = result.coefficients.data(1) * test_dataset.X(::, 4) + result.coefficients.data(0)

simple_model_predictions: DenseVector[Double] = DenseVector(0.1829656516878164, 0.03265205463391061, 0.03265205463391061, 0.03265205463391061, 0.03265205463391061, 0.03265205463391061, 0.09945809776897985, 0.04935356541767792, 0.03265205463391061, 0.03265205463391061, 0.04935356541767792, 0.06605507620144524, 0.04935356541767792, 0.24977169482288564, 0.08275658698521254, 0.03265205463391061, 0.03265205463391061, 0.13286111933651448, 0.06605507620144524, 0.03265205463391061, 0.6005034212819992, 0.03265205463391061, 0.04935356541767792, 0.1662641409040491, 0.5002943565793954, 0.03265205463391061, 0.08275658698521254, 0.03265205463391061, 0.8343245722547415, 0.03265205463391061, 0.3499807595254895, 0.750817018335905, 0.13286111933651448, 0.04935356541767792, 0.03265205463391061, 0.03265205463391061, 0.11615960855274718, 0.6673094644170684, 0.03265205463391061, 0.03265205463391061, 0.03265205463391061, 0.13286111933651448, 0.03265205463391061, 0.06605507620144524, 0.750817018335905, 0.0326

Реализуем функции для расчета вышеобозначенных метрик для пары произвольных векторов.

In [41]:
def r2_of_vectors(y: DenseVector[Double], t: DenseVector[Double]): Double = {
    val length = t.size
    1 - (1.0 / length) * sum((y - t) *:* (y - t)) / breeze.stats.meanAndVariance(y).variance
}

defined function r2_of_vectors

In [42]:
def RMSE_of_vectors(y: DenseVector[Double], t: DenseVector[Double]): Double = {
    val length = t.size
    sqrt( sum((y - t) *:* (y - t)) / length )
}

defined function RMSE_of_vectors

In [43]:
val R2 = r2_of_vectors(simple_model_predictions, test_dataset.Y)
val RMSE = RMSE_of_vectors(simple_model_predictions, test_dataset.Y)

R2: Double = 0.426896068850218
RMSE: Double = 0.1993142635334135

Видно, что коэффициент детерминации получился почти на 30% меньше, чем в предыдущей самописной модели, использующей все признаки (0.42 против 0.66). Величина ``RMSE`` также выше почти вдвое (0.19 против 0.11). Таким образом, один признак в задаче определяет примерно половину значений метрик качества, вторая половина достигается при учете все остальных признаков.

#### Выводы:

В процессе выполнения работы был загружен датасет, содержащий информацию о комментариях пользователей Facebook. Был выполнен разведочный анализ скачанных данных. Датасет был исследован на наличие пропусков в данных. Часть признаков содержала пропуски, они были заменены определенными значениями в соответствии с типом признака. Категориальные признаки были закодированы one-hot-способом. Для количественных признаков были построены гистограммы их распределений, которые позволили выявить наличие выбросов в данных. Для обработки выбросов для каждого признака был построен интервал, содержащий подавляющее количество значений, а все элементы, выходящие за его пределы, принудительно были приравнены к значениям соответствующих концов интервала. Это позволило ограничить область изменения значений признаков шириной интервала, которая затем была преобразована в отрезок [0..1] посредством minMax-масштабирования. Повторная визуализация преобразованных и отмасштабированных признаков продемонстрировала, что признаки получили одинаковые границы, а их распределения стали более подходящими для обучения линейных моделей. Полученные данные были разделены на тренировочную и тестовую части. Тренировочная часть использовались для построения двух моделей линейной регрессии. Первая из них была самописной, реализовывала обучение коэффициентов модели методом градиентного спуска. Вторая представляла собой объект ``leastSquares`` библиотеки Breeze, реализующий регрессию только для одного признака. Данный признак был отобран как имеющий максимальную корреляцию с целевым вектором. Обе модели были успешно обучены, для обеих моделей были рассчитаны метрики качества (коэффициент детерминации $R^2$ и ``RMSE``) на тестовой части датасета. Анализ результатов показал, что первая модель по значению метрик качества почти в два раза превосходит вторую, что объясняется значительно меньшей предсказательной способностью второй модели, учитывающий только один признак в данных.

Работа выполнялась с использованием незнакомого автору до текущего времени языка Scala. В процессе выполнения работы были изучены основные конструкции языка, выполнено знакомство с функциональным стилем программирования, получены начальные навыки работы с данным языком.